<a href="https://colab.research.google.com/github/Brian342/Real-Time-Smoke-Fire-Detection-YOLO11/blob/master/Fire_smoke_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install roboflow

In [4]:
# import all the libraries
import ultralytics
from ultralytics import YOLO
from IPython.display import Image
from roboflow import Roboflow
from google.colab import userdata
import os

ultralytics.checks()

Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/112.6 GB disk)


In [10]:
api_key = userdata.get('ROBOFLOW_API_KEY')

rf = Roboflow(api_key=api_key)
project = rf.workspace("sayed-gamall").project("fire-smoke-detection-yolov11")
dataset1 = project.version(2).download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [11]:
dataset1.location

'/content/Fire-Smoke-Detection-Yolov11-2'

In [12]:
import yaml

file_path = f'{dataset1.location}/data.yaml'

with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

base_path = dataset1.location
data['train'] = f"{base_path}/train/images"
data['val'] = f"{base_path}/valid/image"
data['test'] = f"{base_path}/test/image"

with open(file_path, 'w') as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("Path updated successfully")

Path updated successfully


In [13]:
# verification
from pprint import pprint
with open(file_path, 'r') as file:
    updated_data = yaml.safe_load(file)
    pprint(updated_data)

{'names': ['Fire', 'Smoke'],
 'nc': 2,
 'roboflow': {'license': 'CC BY 4.0',
              'project': 'fire-smoke-detection-yolov11',
              'url': 'https://universe.roboflow.com/sayed-gamal-kn2wu/fire-smoke-detection-yolov11/dataset/2',
              'version': 2,
              'workspace': 'sayed-gamal-kn2wu'},
 'test': '/content/Fire-Smoke-Detection-Yolov11-2/test/image',
 'train': '/content/Fire-Smoke-Detection-Yolov11-2/train/images',
 'val': '/content/Fire-Smoke-Detection-Yolov11-2/valid/image'}


In [17]:
# label cleaning
import os
from collections import defaultdict

def read_yolo_annotations_with_metadata(dataset_metadata):
    # Extract paths and class names
    train_path = dataset_metadata['train'].replace('images', 'labels')
    val_path = dataset_metadata['val'].replace('images', 'labels')
    test_path = dataset_metadata['test'].replace('images', 'labels')
    class_names = dataset_metadata['names']

    # Storage variables
    split_stats = {
        'train': defaultdict(int),
        'val': defaultdict(int),
        'test': defaultdict(int)
    }

    def process_labels(label_dir, split_name):
        for label_file in os.listdir(label_dir):
            if not label_file.endswith(".txt"):
                continue

            label_path = os.path.join(label_dir, label_file)

            # Read annotations
            with open(label_path, 'r') as f:
                lines = [line.strip() for line in f.readlines() if line.strip()]

            # Find present classes
            present_classes = set()
            for line in lines:
                class_id = int(line.strip().split()[0])
                present_classes.add(class_id)

            # Update counts
            if 0 in present_classes and 1 in present_classes:
                split_stats[split_name]['both'] += 1
            elif 0 in present_classes:
                split_stats[split_name]['fire_only'] += 1
            elif 1 in present_classes:
                split_stats[split_name]['smoke_only'] += 1
            else:
                split_stats[split_name]['background'] += 1

    # Process all datasets
    process_labels(train_path, 'train')
    process_labels(val_path, 'val')
    process_labels(test_path, 'test')

    return {
        'class_stats': split_stats,
        'class_names': class_names
    }

# Example usage
dataset_metadata = {
    'names': ['Fire', 'Smoke'],
    'nc': 2,
    'test': '/content/Fire-Smoke-Detection-Yolov11-2/test/images',
    'train': '/content/Fire-Smoke-Detection-Yolov11-2/train/images',
    'val': '/content/Fire-Smoke-Detection-Yolov11-2/valid/images'
}

results = read_yolo_annotations_with_metadata(dataset_metadata)

# Print the report
print("Fire/Smoke Distribution Across Splits:\n")
for split in ['train', 'val', 'test']:
    stats = results['class_stats'][split]
    total = sum(stats.values())
    print(f"{split.upper()} SET ({total} images)")
    print(f"- Fire only: {stats['fire_only']} ({stats['fire_only']/total:.1%})")
    print(f"- Smoke only: {stats['smoke_only']} ({stats['smoke_only']/total:.1%})")
    print(f"- Both present: {stats['both']} ({stats['both']/total:.1%})")
    print(f"- Background (neither): {stats['background']} ({stats['background']/total:.1%})")
    print()

Fire/Smoke Distribution Across Splits:

TRAIN SET (9156 images)
- Fire only: 5963 (65.1%)
- Smoke only: 1674 (18.3%)
- Both present: 1515 (16.5%)
- Background (neither): 4 (0.0%)

VAL SET (872 images)
- Fire only: 598 (68.6%)
- Smoke only: 127 (14.6%)
- Both present: 147 (16.9%)
- Background (neither): 0 (0.0%)

TEST SET (435 images)
- Fire only: 274 (63.0%)
- Smoke only: 79 (18.2%)
- Both present: 82 (18.9%)
- Background (neither): 0 (0.0%)



In [ ]:
import yaml
import os

# Define the YAML file path
yaml_path = f"{dataset.location}/data.yaml"

# Load the current YAML content
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Update the paths to be absolute (this fixes the "Missing Path" error)
data['train'] = os.path.join(dataset.location, 'train', 'images')
data['val'] = os.path.join(dataset.location, 'valid', 'images')
data['test'] = os.path.join(dataset.location, 'test', 'images')

# Save the corrected YAML
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print("data.yaml paths updated to absolute paths!")
print(f"Train path: {data['train']}")
print(f"Val path: {data['val']}")

In [ ]:
# custom dataset
# Start YOLO11 training
!yolo task=detect mode=train \
data={dataset.location}/data.yaml \
model=yolo11n.pt \
epochs=250 \
imgsz=640 \
batch=32 \
patience=20 \
project=fire_detection \
name=smoke_model

Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Fire-Smoke-Detection-Yolov11-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=smoke_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspect

In [ ]:
# Examine Training Results
Image("/kaggle/working/runs/detect/train/confusion_matrix.png", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/labels.jpg", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/results.png", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/train_batch0.jpg", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/val_batch0_pred.jpg", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/val_batch1_pred.jpg", width=600)

In [ ]:
!yolo task=detect mode=train \
data={dataset.location}/data.yaml \
model=yolo11n.pt \
epochs=250 \
imgsz=640 \
batch=32 \
patience=20 \
project=fire_detection \
name=smoke_model

!yolo task=detect mode=val model="/kaggle/working/runs/detect/train/weights/best.pt" data={dataset.location}/data.yaml